In [1]:
import pandas as pd
import os
import glob
import concurrent.futures
import time

directory = 'C:\\Users\\Trieu Pham\\Desktop\\230105 Payment Summary - 2023-01-06T085450.177'
directory2 = 'C:\\Users\\trieu.pham\\Desktop\\230105 Payment Summary - 2023-01-06T085450.177'
cate_dir = 'C:\\Users\\Trieu Pham\\Desktop\\DataCate.xlsx'
cate_dir2 = 'C:\\Users\\trieu.pham\\Desktop\\DataCate.xlsx'

# list of Excel files to read data from
if os.path.exists(os.path.join(directory, '230101 Payment Summary - 2023-01-03T093340.600.xlsx')):
    excel_files = glob.glob(os.path.join(directory, '*.xlsx'))
else:
    excel_files = glob.glob(os.path.join(directory2, '*.xlsx'))
try:
    # use cols 0, Barcode; 2, Category Name
    df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,2])
    pass
except FileNotFoundError:
    df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,2])
    pass
# create an empty list to store the dataframes
# dfs = [] sho

# def excel2csv
def excel2csv(file):
    file_path = 'data.csv'
    # Check if exist file_path
    if not os.path.exists(file_path):
        with open(file_path, 'w') as f:
            pass

    # Read the first line of the file
    with open(file_path, 'r') as f:
        first_line = f.readline().strip()
    # Check if the first line contains the expected column names
    has_headers = 'InvoiceID' in first_line
    df = pd.read_excel(file, sheet_name='Sheet2', skiprows=2, usecols=[4,7])
    # df2 Category
    
    # merge with Category, on "Barcode"
    df = df.merge(df2, on='Barcode', how='left')
    
    # Check if the first line contains the expected column names
    df.to_csv('data.csv', mode='a', header= not has_headers, index=False)


# loop through the list of Excel files
with concurrent.futures.ThreadPoolExecutor() as executor:
    [executor.submit(excel2csv, file) for file in excel_files]
    # for file in excel_files:
    #     executor.submit(excel2csv,file)

# # concatenate the dataframes into a single dataframe
# result = pd.concat(dfs)
# # merge with Category, on "Barcode"
# result = result.merge(df2, on='Barcode', how='left')
# # export the resulting dataframe to a CSV file
# result.to_csv('data.csv', index=False)


In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Sample data
df1 = pd.read_csv('data.csv')


# Get data from Invoice ID with Barcode
df1 = df1[['InvoiceID','Category Name']]

df1 = df1[df1.groupby('InvoiceID')['Category Name'].transform('size') > 1]
dataset_str = df1
dataset_str = df1.groupby('InvoiceID').agg({
    'Category Name': lambda x: ','.join(str(i) for i in x)
}).reset_index()

C:\Users\Trieu Pham\AppData\Local\Temp\ipykernel_9520\3129530308.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('data.csv')


In [2]:
# Transform the data into a list of lists (transactions) with TransactionEncoder
# create a TransactionEncoder object
data = dataset_str[['Category Name']]

data = data.values.tolist()

data = [x[0].split(',') for x in data]

te = TransactionEncoder()

# transform the data into a list of lists
transactions = te.fit(data).transform(data)
transactions = pd.DataFrame(transactions, columns=te.columns_)

transactions

,Accessories,Add-on item Cat,Babycare,Batteries,Biscuits,Bottled Water,Bread Offsite,Candy Category,Carbonated Drinks - CSD,Category Name,...,Supplement,Tea,Tofu,Toys,Vegetables Cat,Vegetarian/Organic food,Whole Milk,Wine,Yogurt,nan
0,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,False,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3682224,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3682225,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3682226,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3682227,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
# Calculate frequent itemsets using Apriori algorithm

frequent_itemsets = apriori(transactions, min_support=0.01, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.034007,(Biscuits)
1,0.168422,(Bottled Water)
2,0.051026,(Bread Offsite)
3,0.049357,(Candy Category)
4,0.107523,(Carbonated Drinks - CSD)
...,...,...
86,0.013239,"(Rice Offsite, Other Cuisine)"
87,0.012438,"(Tea, Other Cuisine)"
88,0.011447,"(Tea, Potato Chips)"
89,0.023905,"(Tea, Rice Offsite)"


In [5]:
# Generate association rules from the frequent itemsets

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Carbonated Drinks - CSD),(Bottled Water),0.107523,0.168422,0.014392,0.133851,0.794732,-0.003717,0.960086
1,(Bottled Water),(Carbonated Drinks - CSD),0.168422,0.107523,0.014392,0.085452,0.794732,-0.003717,0.975867
2,(Bottled Water),(Cold Beverages),0.168422,0.119917,0.013648,0.081034,0.675754,-0.006549,0.957689
3,(Cold Beverages),(Bottled Water),0.119917,0.168422,0.013648,0.113812,0.675754,-0.006549,0.938376
4,(Energy Drinks),(Bottled Water),0.080923,0.168422,0.011107,0.137252,0.814928,-0.002522,0.963871
...,...,...,...,...,...,...,...,...,...
81,(Potato Chips),(Tea),0.066608,0.169312,0.011447,0.171854,1.015018,0.000169,1.003070
82,(Tea),(Rice Offsite),0.169312,0.149482,0.023905,0.141190,0.944529,-0.001404,0.990345
83,(Rice Offsite),(Tea),0.149482,0.169312,0.023905,0.159920,0.944529,-0.001404,0.988820
84,(Yogurt),(Rice Offsite),0.075173,0.149482,0.011032,0.146749,0.981718,-0.000205,0.996797


In [6]:
# Display the association rules with support, lift, and confidence

print(rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']])
rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']].to_csv('result.csv', index=False)

                  antecedents                consequents   support      lift  \
0   (Carbonated Drinks - CSD)            (Bottled Water)  0.014392  0.794732   
1             (Bottled Water)  (Carbonated Drinks - CSD)  0.014392  0.794732   
2             (Bottled Water)           (Cold Beverages)  0.013648  0.675754   
3            (Cold Beverages)            (Bottled Water)  0.013648  0.675754   
4             (Energy Drinks)            (Bottled Water)  0.011107  0.814928   
..                        ...                        ...       ...       ...   
81             (Potato Chips)                      (Tea)  0.011447  1.015018   
82                      (Tea)            (Rice  Offsite)  0.023905  0.944529   
83            (Rice  Offsite)                      (Tea)  0.023905  0.944529   
84                   (Yogurt)            (Rice  Offsite)  0.011032  0.981718   
85            (Rice  Offsite)                   (Yogurt)  0.011032  0.981718   

    confidence  
0     0.133851  
1    